#José Sebastián Naranjo Zamudio A01066941

# AI. Actividad Integradora

## Instrucciones

•Realiza la siguiente simulación:
* Inicializa las posiciones iniciales de las Kcajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
* Todos los agentes empiezan en posición aleatorias vacías.
* Se ejecuta el tiempo máximo establecido.

•Deberás recopilar la siguiente información durante la ejecución:
* Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
* Número de movimientos realizados por todos los robots.
* Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.

## Imports

In [ ]:
!pip3 install mesa

In [9]:
from mesa import Agent, Model 
from mesa.space import Grid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime



In [10]:
class Move(Agent):
    #Inicializacion
    grid = None
    x = None
    y = None
    moore = False
    include_center=False

    def __init__(self, unique_id, pos, model, moore=False): #moore es para moverse en 8 direcciones
    
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore
    
    def move(self):
        next_moves = self.model.grid.get_neighborhood(self.pos,moore=False,include_center=False)
        posicion_azar=random.choice(next_moves)
        if self.model.grid.is_cell_empty(posicion_azar):
            self.movimientos += 1
            self.model.grid.move_agent(self, posicion_azar)

## Crear el modelo

In [11]:
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        
        if isinstance(contenido_celda,Robot):
            habitacion[x][y] = 6
             
        elif isinstance(contenido_celda, Caja):
            habitacion[x][y]=contenido_celda.estiba
    return habitacion
    
class Robot(Move):
    
    def __init__(self,ide, pos, modelo):
        super().__init__(ide,pos, modelo)
        self.movimientos = 0
        self.box_carry=False
        self.boxes=0
        
    def step(self):
        
        self.move()
        vecinos = self.model.grid.get_neighbors(self.pos,moore=False,include_center=False,radius=1)
        for vecino in vecinos:
            if isinstance(vecino, Caja): #si los vecinos son cajas
                if self.boxes<4: #si no estoy cargando caja
                    if vecino.estiba == 1: #celda con una caja
                        #self.box_carry=True #carga la caja
                        vecino.estiba=0     #caja se va
                        self.boxes+=1
                        break
                        
                elif self.boxes==4: #si lleva una caja
                    if vecino.estiba >= 1 and vecino.estiba < 5: #si una celda tiene una caja o menos de cinco
                        vecino.estiba += 4 #se le suma a la estiba
                        self.box_carry=False #ya no llevo la caja
                        self.boxes=0
        
                        break
                else:
                    self.move()
          

class Caja(Agent):
    
    def __init__(self, pos, modelo):
        super().__init__(pos, modelo)
        self.estiba= 1
        self.x, self.y = pos
        

class Warehouse(Model):
    
    def __init__(self, m, n, num_agentes, num_cajas):
        self.num_agentes = num_agentes
        self.num_cajas = num_cajas
        self.grid = Grid(m, n, True)
        self.schedule = SimultaneousActivation(self)
           
        # Posicionar cajas aleatoriamente
        celdas_sucias = num_cajas
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in range(celdas_sucias):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Caja(celda_vacia, self)
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
       
        # Posicionar agentes robots
        lista_celdas_vacias = list(self.grid.empties)
        for i in range(num_agentes):
            celda_vacia_robot=random.choice(lista_celdas_vacias)
            robot = Robot(i,celda_vacia_robot, self)
            self.grid.place_agent(robot, celda_vacia_robot)
            self.schedule.add(robot)
            lista_celdas_vacias.remove(celda_vacia_robot)
            
        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': obtener_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)})
        
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todas_cajas_apiladas(self):
        estibas_posibles=self.num_cajas//5
        global estibas_hechas
        estibas_hechas = 0
        global estibas_incompletas 
        estibas_incompletas=0
        global cajas_solas
        cajas_solas=0

        if estibas_posibles == 0:
            print("Cajas insuficientes para el modelo")
            return False
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            if isinstance(contenido_celda, Caja) and celda[0].estiba ==1:
                cajas_solas += 1
            if isinstance(contenido_celda, Caja) and celda[0].estiba < 5 and celda[0].estiba >1:
                estibas_incompletas += 1
            if isinstance(contenido_celda, Caja) and celda[0].estiba == 5:
                estibas_hechas += 1
        if estibas_hechas == estibas_posibles:
            return True
        else:
            return False


Correr el modelo

In [12]:
# Datos de la habitacion:
M = 15
N = 15

# Numero de agentes
NUM_AGENTES =5

# Porcentaje de celdas inicialmente sucias:
Cantidad_Cajas =30

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Warehouse(M, N, NUM_AGENTES, Cantidad_Cajas)

while(time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todas_cajas_apiladas(): #termina cuando se lleguen a las estibas maximas posibles
    modelo.step()
    #print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización

In [13]:
all_grid = modelo.colectordatos.get_model_vars_dataframe()

In [14]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='YlOrBr')
plt.colorbar()

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [15]:
anim

## Informe

In [16]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()
print('Tiempo necesario hasta maximas estibas:', tiempo_ejecucion, '/', tiempo_inicio)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())
print('Estibas completas = ', estibas_hechas)
print('Estibas incompletas = ', estibas_incompletas)
print('Cajas Solas = ',cajas_solas)

Tiempo necesario hasta maximas estibas: 0:00:00.060323 / 0:00:00.060000
Número de movimientos realizados por todos los agentes: 354.0
Estibas completas =  3
Estibas incompletas =  0
Cajas Solas =  5


## Conclusiones:
* Se puede mejorar el tiempo si los robots poseen un espacio determinado donde dejar las cajas.
* Tambien de podria dividir el espacio entre areas de accion, donde cada robot trabajara con sus cajas y formará sus estibas
En este caso la creacion de las estibas depende mucho de cuestiones de azar ya que los robots tienen conocimiento limitado
de su entorno por lo tanto las decisiones tomadas son a corto plazo y de alcance muy pequeño.

En resolución, la manera mas sencilla y con mejores resultados es si los agentes ya poseen un conocimiento previo a la corrida
del modelo,e inclusive un sistema centinela que los puediera coordinar en tiempo real, de esta manera es posible que los agentes pueden optar por la ruta mas optima y pueden tambien realizar un mejor
acomodo de las cajas,ya que de otra manera es muy probable que se obtengan estibas incompletas. Si no es posible aplicar lo anterior se pueden tomar en cuanta las recomendaciones mencionadas arriba.
